In [23]:
import sys
sys.path.append('../')
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from plotting import CandlePlot

In [24]:
df_raw = pd.read_pickle("../data/GBP_JPY_H1.pkl")

In [25]:
df_raw.shape

(68406, 14)

In [26]:
df_an = df_raw.copy() #df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [27]:
df_an = RSI(df_an)

In [ ]:
df_an = apply_patterns(df_an)

In [ ]:
df_an['EMA_200'] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [ ]:
df_an.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'RSI_14',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'EMA_200'],
      dtype='object')

In [ ]:
our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'direction', 'ENGULFING', 'EMA_200', 'RSI_14']

In [ ]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [ ]:
df_slim.tail()

,time,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,direction,ENGULFING,EMA_200,RSI_14
68202,2025-06-27 19:00:00+00:00,198.193,198.428,198.180,198.412,198.177,198.412,198.164,198.395,198.209,198.447,198.195,198.429,1,False,197.233094,51.435464
68203,2025-06-27 20:00:00+00:00,198.410,198.506,198.344,198.467,198.392,198.472,198.322,198.420,198.427,198.541,198.363,198.514,1,False,197.245371,53.058522
68204,2025-06-29 21:00:00+00:00,197.921,198.294,197.916,198.206,197.696,198.119,197.691,197.981,198.146,198.510,198.128,198.431,1,False,197.254930,45.318336
68205,2025-06-29 22:00:00+00:00,198.167,198.466,198.167,198.298,197.942,198.444,197.942,198.278,198.392,198.488,198.277,198.319,1,False,197.265309,48.187547
68206,2025-06-29 23:00:00+00:00,198.291,198.319,198.188,198.240,198.267,198.296,198.165,198.218,198.315,198.342,198.211,198.262,-1,False,197.275007,46.529953


In [ ]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.mid_h > row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE
            

In [ ]:
df_slim['SIGNAL'] = df_slim.apply(apply_signal, axis=1)

In [ ]:
df_slim['SIGNAL'].value_counts()

SIGNAL
 0    62839
 1     3431
-1     1937
Name: count, dtype: int64

In [ ]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5

def apply_take_profit(row):
    if row.SIGNAL != NONE:
        if row.SIGNAL == BUY:
            return (row.ask_c - row.ask_o) * PROFIT_FACTOR + row.ask_c
        else:
            return (row.bid_c - row.bid_o) * PROFIT_FACTOR + row.bid_c
    else:
        return 0.0
    
def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        if row.SIGNAL == BUY:
            return row.ask_o
        else:
            return row.bid_o
        
    else:
        return 0.0

In [ ]:
df_slim['TP'] = df_slim.apply(apply_take_profit, axis=1)
df_slim['SL'] = df_slim.apply(apply_stop_loss, axis=1)

In [ ]:
df_slim[df_slim.SIGNAL == BUY].head()

,time,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,direction,ENGULFING,EMA_200,RSI_14,SIGNAL,TP,SL
82,2014-07-15 17:00:00+00:00,174.250,174.363,174.219,174.356,174.239,174.351,174.207,174.345,174.262,174.375,174.230,174.367,1,True,173.922715,67.754239,1,174.5245,174.262
86,2014-07-15 21:00:00+00:00,174.294,174.346,174.256,174.312,174.251,174.318,174.199,174.288,174.337,174.375,174.302,174.337,1,True,173.940008,64.503196,1,174.3370,174.337
96,2014-07-16 07:00:00+00:00,174.219,174.351,174.167,174.351,174.208,174.341,174.157,174.339,174.230,174.364,174.177,174.363,1,True,173.977560,60.362914,1,174.5625,174.230
107,2014-07-16 18:00:00+00:00,174.227,174.306,174.226,174.274,174.215,174.296,174.215,174.265,174.239,174.318,174.237,174.284,1,True,174.009225,53.184836,1,174.3515,174.239
347,2014-07-30 18:00:00+00:00,173.940,174.234,173.820,174.020,173.856,174.202,173.759,174.009,174.023,174.288,173.851,174.032,1,True,173.158872,79.982292,1,174.0455,174.023


In [ ]:
df_plot = df_slim.iloc[2068:2080]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[cp.df_plot.SIGNAL != NONE]
markers = ['mid_c', 'TP', 'SL']
marker_colors = ["#0000FF", "#00FF00", "#FF0000"]
for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x = trades.sTime,
        y = trades[markers[i]],
        mode = 'markers',
        marker = dict(color=marker_colors[i], size=12)
    ))
cp.show_plot(line_traces=["EMA_200"], sec_traces=['RSI_14'])

In [ ]:
class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name

        if row.SIGNAL == BUY:
            self.start_price = row.ask_c
            self.trigger_price = row.ask_c
        else:
            self.start_pirce = row.bid_c
            self.trigger_price = row.bid_c

        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 0
        
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
        
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.bid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.bid_h)
            elif row.bid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.bid_l)
        if self.SIGNAL == SELL:
            if row.ask_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.ask_l)
            elif row.ask_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.ask_h)    

In [ ]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
        open_trades = [x for x in open_trades if x.running == True]

    if row.SIGNAL != NONE:
        open_trades.append(Trade(row))

In [ ]:
df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [ ]:
df_results.result.sum()

np.float64(-863.0)

In [ ]:
df_results.sort_values(by="start_index", inplace=True)

In [ ]:
df_m5 = pd.read_pickle("../data/EUR_USD_M5.pkl")

In [ ]:
df_m5.shape

(818772, 14)

In [ ]:
df_m5.time.max()

Timestamp('2025-06-29 23:55:00+0000', tz='tzutc()')

In [ ]:
df_raw.time.max()

Timestamp('2025-06-29 23:00:00+0000', tz='tzutc()')

In [ ]:
from dateutil import parser

In [ ]:
time_min = parser.parse("2021-12-15T10:00:00Z")
time_max = parser.parse("2021-12-15T11:00:00Z")
df_m5_s = df_m5[(df_m5.time>=time_min)&(df_m5.time<=time_max)]
df_raw_s = df_raw[(df_raw.time>=time_min)&(df_raw.time<=time_max)]

In [ ]:
df_m5_s

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
554748,2021-12-15 10:00:00+00:00,208,1.12716,1.12745,1.12709,1.12734,1.12708,1.12738,1.12702,1.12727,1.12723,1.12752,1.12715,1.12742
554749,2021-12-15 10:05:00+00:00,132,1.12733,1.12754,1.12726,1.12746,1.12726,1.12747,1.12719,1.12739,1.12740,1.12760,1.12733,1.12752
554750,2021-12-15 10:10:00+00:00,166,1.12744,1.12752,1.12732,1.12746,1.12737,1.12745,1.12725,1.12739,1.12750,1.12760,1.12739,1.12753
554751,2021-12-15 10:15:00+00:00,138,1.12744,1.12770,1.12740,1.12761,1.12737,1.12763,1.12733,1.12754,1.12752,1.12776,1.12747,1.12768
554752,2021-12-15 10:20:00+00:00,252,1.12760,1.12762,1.12712,1.12712,1.12753,1.12756,1.12705,1.12705,1.12767,1.12769,1.12719,1.12719
554753,2021-12-15 10:25:00+00:00,198,1.12711,1.12726,1.12695,1.12695,1.12704,1.12719,1.12688,1.12688,1.12718,1.12733,1.12702,1.12702
554754,2021-12-15 10:30:00+00:00,142,1.12696,1.12698,1.12680,1.12696,1.12689,1.12691,1.12674,1.12690,1.12703,1.12706,1.12686,1.12703
554755,2021-12-15 10:35:00+00:00,121,1.12698,1.12712,1.12698,1.12700,1.12691,1.12705,1.12691,1.12693,1.12704,1.12718,1.12704,1.12706
554756,2021-12-15 10:40:00+00:00,206,1.12698,1.12707,1.12651,1.12656,1.12692,1.12700,1.12644,1.12649,1.12705,1.12714,1.12658,1.12663
554757,2021-12-15 10:45:00+00:00,165,1.12655,1.12668,1.12645,1.12654,1.12648,1.12661,1.12638,1.12648,1.12662,1.12675,1.12652,1.12661


In [ ]:
df_m5_slim = df_m5[['time', 'mid_h', 'mid_l']].copy()

In [ ]:
df_m5_slim.head()

,time,mid_h,mid_l
0,2014-06-30 00:00:00+00:00,1.36440,1.36426
1,2014-06-30 00:05:00+00:00,1.36454,1.36434
2,2014-06-30 00:10:00+00:00,1.36471,1.36450
3,2014-06-30 00:15:00+00:00,1.36457,1.36436
4,2014-06-30 00:20:00+00:00,1.36450,1.36437


In [ ]:
import datetime as dt

In [ ]:
df_signals = df_slim = df_slim[df_slim.SIGNAL != NONE].copy()

In [ ]:
df_signals.head()

,time,mid_o,mid_h,mid_l,mid_c,ask_c,bid_c,direction,ENGULFING,EMA_200,RSI_14,SIGNAL,TP,SL
82,2014-07-15 17:00:00+00:00,174.250,174.363,174.219,174.356,174.367,174.345,1,True,173.922715,67.754239,1,174.5150,174.250
86,2014-07-15 21:00:00+00:00,174.294,174.346,174.256,174.312,174.337,174.288,1,True,173.940008,64.503196,1,174.3390,174.294
96,2014-07-16 07:00:00+00:00,174.219,174.351,174.167,174.351,174.363,174.339,1,True,173.977560,60.362914,1,174.5490,174.219
97,2014-07-16 08:00:00+00:00,174.346,174.438,174.010,174.165,174.177,174.153,-1,True,173.979526,49.557090,-1,173.8935,174.346
107,2014-07-16 18:00:00+00:00,174.227,174.306,174.226,174.274,174.284,174.265,1,True,174.009225,53.184836,1,174.3445,174.227


In [ ]:
df_signals['m5_start'] = [x + dt.timedelta(hours=1) for x in df_signals.time]

In [ ]:
df_signals['start_index_h1'] = df_signals.index

In [ ]:
df_signals.drop(['time', 'mid_o', 'mid_l', 'mid_h', 'direction', 'ask_c', 'bid_c', 'ENGULFING', 'EMA_200', 'RSI_14'], axis=1, inplace=True)

In [ ]:
df_signals.head()

,mid_c,SIGNAL,TP,SL,m5_start,start_index_h1
82,174.356,1,174.5150,174.250,2014-07-15 18:00:00+00:00,82
86,174.312,1,174.3390,174.294,2014-07-15 22:00:00+00:00,86
96,174.351,1,174.5490,174.219,2014-07-16 08:00:00+00:00,96
97,174.165,-1,173.8935,174.346,2014-07-16 09:00:00+00:00,97
107,174.274,1,174.3445,174.227,2014-07-16 19:00:00+00:00,107


In [ ]:
df_signals.rename(columns={
    'mid_c': 'start_price',
    'm5_start': 'time'
}, inplace=True)

In [ ]:
df_signals.head()

,start_price,SIGNAL,TP,SL,time,start_index_h1
82,174.356,1,174.5150,174.250,2014-07-15 18:00:00+00:00,82
86,174.312,1,174.3390,174.294,2014-07-15 22:00:00+00:00,86
96,174.351,1,174.5490,174.219,2014-07-16 08:00:00+00:00,96
97,174.165,-1,173.8935,174.346,2014-07-16 09:00:00+00:00,97
107,174.274,1,174.3445,174.227,2014-07-16 19:00:00+00:00,107


In [ ]:
df_m5_slim.head()

,time,mid_h,mid_l
0,2014-06-30 00:00:00+00:00,1.36440,1.36426
1,2014-06-30 00:05:00+00:00,1.36454,1.36434
2,2014-06-30 00:10:00+00:00,1.36471,1.36450
3,2014-06-30 00:15:00+00:00,1.36457,1.36436
4,2014-06-30 00:20:00+00:00,1.36450,1.36437


In [ ]:
merged = pd.merge(left=df_m5_slim, right=df_signals, on='time', how='left')

In [ ]:
merged.fillna(0, inplace=True)

In [ ]:
merged.SIGNAL = merged.SIGNAL.astype(int)
merged.start_index_h1 = merged.start_index_h1.astype(int)

In [ ]:
merged.head()

,time,mid_h,mid_l,start_price,SIGNAL,TP,SL,start_index_h1
0,2014-06-30 00:00:00+00:00,1.36440,1.36426,0.0,0,0.0,0.0,0
1,2014-06-30 00:05:00+00:00,1.36454,1.36434,0.0,0,0.0,0.0,0
2,2014-06-30 00:10:00+00:00,1.36471,1.36450,0.0,0,0.0,0.0,0
3,2014-06-30 00:15:00+00:00,1.36457,1.36436,0.0,0,0.0,0.0,0
4,2014-06-30 00:20:00+00:00,1.36450,1.36437,0.0,0,0.0,0.0,0


In [ ]:
class TradeM5:
    def __init__(self, row):
        self.running = True
        self.start_index_m5 = row.name
        self.start_index_h1 = row.start_index_h1
        self.start_price = row.start_price
        self.trigger_price = row.start_price
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 1
        
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
        
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.mid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_h)
            elif row.mid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_l)
        if self.SIGNAL == SELL:
            if row.mid_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.mid_l)
            elif row.mid_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.mid_h)    

In [ ]:
open_trades_m5 = []
closed_trades_m5 = []

for index, row in merged.iterrows():
    if row.SIGNAL != NONE:
        open_trades_m5.append(TradeM5(row))
        
    for ot in open_trades_m5:
        ot.update(row)
        if ot.running == False:
            closed_trades_m5.append(ot)
        open_trades_m5 = [x for x in open_trades_m5 if x.running == True]

   

In [ ]:
df_res_m5 = pd.DataFrame.from_dict([vars(x) for x in closed_trades_m5])

In [ ]:
df_res_m5.head()

,running,start_index_m5,start_index_h1,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,duration
0,False,3384,82,174.356,1.35671,1,174.5150,174.250,-1.0,2014-07-15 18:05:00+00:00,2014-07-15 18:00:00+00:00,1
1,False,3432,86,174.312,1.35678,1,174.3390,174.294,-1.0,2014-07-15 22:05:00+00:00,2014-07-15 22:00:00+00:00,1
2,False,3552,96,174.351,1.35521,1,174.5490,174.219,-1.0,2014-07-16 08:05:00+00:00,2014-07-16 08:00:00+00:00,1
3,False,3564,97,174.165,1.35412,-1,173.8935,174.346,1.5,2014-07-16 09:05:00+00:00,2014-07-16 09:00:00+00:00,1
4,False,3684,107,174.274,1.35210,1,174.3445,174.227,-1.0,2014-07-16 19:05:00+00:00,2014-07-16 19:00:00+00:00,1


In [ ]:
df_res_m5.result.sum()

np.float64(-522.5)